# Import packages

In [77]:
from collections import Counter
import re
import spacy
# from spacy_langdetect import LanguageDetector
# from spacy.language import Language
from collections import Counter
# from utils import detect_para

from operator import itemgetter
import fitz

import jellyfish

import numpy as np

## Load Spacy voor taal detectie
wordt momenteel niet gebruikt

In [6]:
def get_lang_detector(nlp, name):
    return LanguageDetector()

nlp = spacy.load("nl_core_news_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)

NameError: name 'Language' is not defined

### Functie om vaakstvoorkomende taal vast te stellen

In [7]:
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]

## Main functies
PDF als input en een lijst van dictionaries als output. 

In [61]:
# functie om te controleren of value een nummer is.
def digitize(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

def fonts(doc, granularity=True):
    """Extracts fonts and their usage in PDF documents.

    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param granularity: also use 'font', 'flags' and 'color' to discriminate text
    :type granularity: bool

    :rtype: [(font_size, count), (font_size, count}], dict
    :return: most used fonts sorted by count, font style information
    """
    styles = {}
    font_counts = {}

    for page in doc:
        blocks = page.get_text("dict", flags=11)["blocks"]
        # print(blocks)
        for b in blocks:  # iterate through the text blocks
            # print(b)
            if b['type'] == 0:  # block contains text
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if granularity:
                            identifier = "{0}_{1}_{2}_{3}".format(s['size'], s['flags'], s['font'], s['color'])
                            styles[identifier] = {'size': s['size'], 'flags': s['flags'], 'font': s['font'],
                                                  'color': s['color']}
                            
                        else:
                            identifier = "{0}".format(s['size'])
                            styles[identifier] = {'size': s['size'], 'font': s['font']}
                            
                        font_counts[identifier] = font_counts.get(identifier, 0) + 1  # count the fonts usage

    font_counts = sorted(font_counts.items(), key=itemgetter(1), reverse=True)

    # print(b, identifier)
    # print(styles)

    if len(font_counts) < 1:
        raise ValueError("Zero discriminating fonts found!")

    return font_counts, styles


def font_tags(font_counts, styles):
    """Returns dictionary with font sizes as keys and tags as value.

    :param font_counts: (font_size, count) for all fonts occuring in document
    :type font_counts: list
    :param styles: all styles found in the document
    :type styles: dict

    :rtype: dict
    :return: all element tags based on font-sizes
    """
    p_style = styles[font_counts[0][0]]  # get style for most used font by count (paragraph)
    p_size = p_style['size']  # get the paragraph's size


    # sorting the font sizes high to low, so that we can append the right integer to each tag
    font_sizes = []
    for (font_size, count) in font_counts:
        font_sizes.append(float(font_size))
    font_sizes.sort(reverse=True)
    
    # aggregating the tags for each font size
    idx = 0
    size_tag = {}
    for size in font_sizes:
        idx += 1
        if size == p_size:
            idx = 0
            size_tag[size] = '<p>'
        if size > p_size:
            size_tag[size] = '<h{0}>'.format(idx)
        elif size < p_size:
            size_tag[size] = '<s{0}>'.format(idx)

    return size_tag


def headers_para(doc, size_tag):
    """Scrapes headers & paragraphs from PDF and return texts with element tags.

    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param size_tag: textual element tags for each size
    :type size_tag: dict

    :rtype: list
    :return: texts with pre-prended element tags
    """
    header_para = []  # list with headers and paragraphs
    first = True  # boolean operator for first header
    previous_s = {}  # previous span
    page_num = 1
    b_ = []
    s_ = []
    t_point = 20 # amount of pixels/points that defines a white line is between two lines.

    for page in doc:
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # this block contains text+

                # REMEMBER: multiple fonts and sizes are possible IN one block, wil ik dit? Of wil ik gewogen? 
                # Hoe vaak kom je titel tegen met in dezelfde lijn paragraaf_text? functie is niet gearceerde woorden eruit te halen.

                block_string = ""  # text found in block
                for l in b["lines"]:  # iterate through the text lines
                    
                    for s in l["spans"]:  # iterate through the text spans
                        s_.append(s)
                        if first:
                            previous_s = s
                            first = False
                            block_string = {
                                'tag': size_tag[s['size']], 
                                'text': s['text'].strip(),
                                'page_num': page_num,
                                'p_position_x': s['origin'][0],
                                'p_position_y': s['origin'][1],
                                'font_size': s['size']
                                }
                            header_para.append(block_string)
                        else:
                            previous_s = header_para[-1]
                            if size_tag[s['size']] == previous_s['tag'] and 0 <= s['origin'][1] - previous_s['p_position_y'] <= (t_point + previous_s['font_size']) and size_tag[s['size']] != '<p>':

                                last_entry = header_para[-1]
                                last_entry['text'] += f" {s['text'].strip()}" 
                                last_entry['p_position_x'] = s['origin'][0]
                                last_entry['p_position_y'] = s['origin'][1]

                                header_para[-1] = last_entry
                            
                            elif size_tag[s['size']] == previous_s['tag'] and size_tag[s['size']] == '<p>':

                                last_entry = header_para[-1]
                                last_entry['text'] += f" {s['text'].strip()}" 
                                last_entry['p_position_x'] = s['origin'][0]
                                last_entry['p_position_y'] = s['origin'][1]

                                header_para[-1] = last_entry
                            
                            elif s['text'].strip() == '':
                                pass

                            else:
                                block_string = {
                                    'tag': size_tag[s['size']], 
                                    'text': s['text'].strip(),
                                    'page_num': page_num,
                                    'p_position_x': s['origin'][0],
                                    'p_position_y': s['origin'][1],
                                    'font_size': s['size']
                                    }
                                header_para.append(block_string)
        if len(header_para) > 0 and digitize(header_para[-1]['text'].strip()): # check if last line on page is a (page) number and remove it.
            header_para.pop(-1)
        
        # het toevoegen van een dictionary die het einde van de pagina aangeeft. Maakt het werken rondom begin en einde van pagian makkelijker.
        block_string = {
            'tag': '<p_break>', 
            'text': f'A physical page break on page {page_num}',
            'page_num': page_num,
            'p_position_x': 0,
            'p_position_y': 0,
            'font_size': 0
            }
        header_para.append(block_string)
        page_num += 1
    return header_para, s_

def detect_para(path):

    doc = fitz.open(path)

    font_counts, styles = fonts(doc, granularity=False)
    # print(font_counts)
    # print(styles)

    size_tag = font_tags(font_counts, styles)
    # print(size_tag)

    elements, b_ = headers_para(doc, size_tag)
    topscript_list = [elements.index(elem)+1 for elem in elements if elem['tag'] == '<p_break>'] # make list of first entries on page
    topscript_list.pop(-1) # remove last, because there is no topscript on the next page after the last page
    elements = pop_repeating(elements, topscript_list)
    endscript_list = [elements.index(elem)-1 for elem in elements if elem['tag'] == '<p_break>'] # make list of last entries on page
    elements = pop_repeating(elements, endscript_list)

    primary_heading = deter_primary_h(elements)
    return elements, b_, primary_heading

def pop_repeating(elements, s_list): # based on list of first or last entries on page, deter if these entries are similair and thus a standard footer for example.
    sim_list = []
    for i in range(len(s_list)-1):
        a, b = s_list[i], s_list[i+1]
        str_dist = str_distance(a, b, elements)
        if str_dist >= 0.8:
            sim_list.extend([a, b])
    sim_list = list(set(sim_list))
    sim_list.sort(reverse=True)
    if len(sim_list) - len(s_list) <= 3:
        [elements.pop(i) for i in sim_list]
    return elements

def str_distance(a, b, elements):
    a = elements[a]['text']
    b = elements[b]['text']
    dist = jellyfish.jaro_distance(a, b)
    return dist

# beter om terug te zoeken vanaf <p> en alle hogere headings een plek geven.
def deter_primary_h(elements):
    b = [elem['tag'] for elem in elements] # make list of only the tags
    b_ = []
    for i in range(len(b)-2): # walk through list for headings that is inbetween 2 paragraphs
        if b[i] == b[i+2] and b[i] == '<p>':
            b_.append(b[i+1])
    tag_amount = []
    
    for x in list(set(b_)):
        if x.startswith('<h'): # only keep headings
            d_tag_amoung = {
                'tag': x,
                'amount': b_.count(x) 
            }
            tag_amount.append(d_tag_amoung)
    tag_amount = sorted(tag_amount, key=lambda i: i['amount'], reverse=True)
    
    primary = tag_amount[0]['tag'] if len(tag_amount) > 0 else ''
    return primary

## Initiatie van PDF minen

In [64]:
paragraph = []
heading = []
lang = []

path = 'Raadsprogramma 2022-2026 Aan de slag voor Aalsmeer.pdf'
# path = "Coalitieakkoord-Samen-werken-aan-Delft.pdf"
path = 'Coalitieakkoord-2022-2026-1.pdf' # Alphen a/d Rijn

elements, b_ ,primary_heading = detect_para(path)


In [65]:
# display of output
elements

[{'tag': '<s3>',
  'text': 'Zichtbaar en dichtbij - Samen bouwen aan een duurzame en vitale gemeente',
  'page_num': 1,
  'p_position_x': 125.21260070800781,
  'p_position_y': 823.3514404296875,
  'font_size': 8.5},
 {'tag': '<s3>',
  'text': '1',
  'page_num': 1,
  'p_position_x': 35.90549850463867,
  'p_position_y': 823.351318359375,
  'font_size': 8.5},
 {'tag': '<h1>',
  'text': 'Zichtbaar en dichtbij',
  'page_num': 1,
  'p_position_x': 122.83470153808594,
  'p_position_y': 214.18701171875,
  'font_size': 63.0},
 {'tag': '<h7>',
  'text': 'Samen bouwen aan een duurzame en vitale gemeente',
  'page_num': 1,
  'p_position_x': 125.21260070800781,
  'p_position_y': 282.14178466796875,
  'font_size': 20.0},
 {'tag': '<h8>',
  'text': 'Coalitieakkoord 2022 - 2026',
  'page_num': 1,
  'p_position_x': 125.21260070800781,
  'p_position_y': 61.2662353515625,
  'font_size': 18.0},
 {'tag': '<p_break>',
  'text': 'A physical page break on page 1',
  'page_num': 1,
  'p_position_x': 0,
  'p_po

In [40]:
# HEADING LOGIC
####
# heading above paragraph == head of paragraph
# biggest heading on page == page heading/title (?)
# biggest heading == titel (?)
# when len(heading) > 10, dan geen echte heading maar inleidend stuk 

In [41]:
# FINAL OUTPUT
####
# Paragraph text
# Direct heading
# as dict or as HTML?
# document content based on headings

In [42]:
# TODO natural place on page
#for page in doc:
#        blocks = page.get_text("dict")["blocks"]
#        for b in blocks:  # iterate through the text blocks
#            if b['type'] == 0:  # this block contains text+
#
#                # REMEMBER: multiple fonts and sizes are possible IN one block, wil ik dit? Of wil ik gewogen? 
#                # Hoe vaak kom je titel tegen met in dezelfde lijn paragraaf_text? functie is niet gearceerde woorden eruit te halen.#
#
#                block_string = ""  # text found in block
#                for l in b["lines"]:  # iterate through the text lines
#                    
#                    for s in l["spans"]: 

logic of bbox:
- first entry: (112.77839660644531, -> start x-axis (so logically left-upper) 
- second: 123.299072265625, -> start y-axis (so logically left-upper) 
- third: 316.4429931640625, -> end x-axis (so logically right-under) 
- fourth: 145.89569091796875) -> end y-axis (so logically right-under) 

In [107]:
def relative_borderdistance(box, x_page, y_page):
    x_l_distance = round(box[0] / x_page, 3)
    x_r_distance = round(box[2] / x_page, 3)
    y_t_distance = round(box[1] / y_page, 3)
    y_b_distance = round(box[3] / y_page, 3)
    return y_t_distance, y_b_distance, x_l_distance, x_r_distance, 


doc = fitz.open(path)
page = 3
# page in doc, block on page, for b in block, for line in b, for s in line
blocks = doc[page].get_text("dict")["blocks"] # ["blocks"] piece where information is stored
x_page = doc[page].get_text("dict")['width']
y_page = doc[page].get_text("dict")['height']
print(x_page, y_page)
list_of_boxes = [block['bbox'] for block in blocks]
rel_x_dist = []
y = [relative_borderdistance(box, x_page, y_page) for box in list_of_boxes]
x = np.array(y)

# elk document heeft een marge, maar niet elk document is deze hetzelfde. Het is handig om te weten welke tekst als 
# header en footer gezien kan worden en welke volgorde het document gelezen moet worden. Het is echter gevaarlijk om 
# een vast percentage aan te wijzen als waarschijnlijkheid voor de marge. 

#### 
# om de marge te herkennen moeten de dozen op een pagina allemaal bekeken worden, gekeken worden of de min en max 
# vaker voorkomen, en dus bepalen of er überhaupt een header en footer is en of alle text niet extreem strak langs 
# de randen van de pagina is uitgeleind.

####


1190.55 841.89


In [108]:
y

[(0.968, 0.981, 0.03, 0.971),
 (0.041, 0.557, -0.001, 0.764),
 (0.745, 0.914, 0.095, 0.277),
 (0.631, 0.701, 0.594, 0.777),
 (0.712, 0.914, 0.594, 0.779),
 (0.607, 0.693, 0.094, 0.422),
 (0.741, 0.914, 0.287, 0.472),
 (0.146, 0.23, 0.787, 0.96),
 (0.242, 0.487, 0.787, 0.971),
 (0.503, 0.572, 0.787, 0.972),
 (0.588, 0.715, 0.787, 0.972),
 (0.726, 0.857, 0.787, 0.969),
 (0.869, 0.914, 0.787, 0.97)]

In [109]:
x[np.lexsort(x.T[::-1])]

array([[ 0.041,  0.557, -0.001,  0.764],
       [ 0.146,  0.23 ,  0.787,  0.96 ],
       [ 0.242,  0.487,  0.787,  0.971],
       [ 0.503,  0.572,  0.787,  0.972],
       [ 0.588,  0.715,  0.787,  0.972],
       [ 0.607,  0.693,  0.094,  0.422],
       [ 0.631,  0.701,  0.594,  0.777],
       [ 0.712,  0.914,  0.594,  0.779],
       [ 0.726,  0.857,  0.787,  0.969],
       [ 0.741,  0.914,  0.287,  0.472],
       [ 0.745,  0.914,  0.095,  0.277],
       [ 0.869,  0.914,  0.787,  0.97 ],
       [ 0.968,  0.981,  0.03 ,  0.971]])

In [104]:
x

array([[ 0.968,  0.981,  0.03 ,  0.971],
       [ 0.041,  0.557, -0.001,  0.764],
       [ 0.745,  0.914,  0.095,  0.277],
       [ 0.631,  0.701,  0.594,  0.777],
       [ 0.712,  0.914,  0.594,  0.779],
       [ 0.607,  0.693,  0.094,  0.422],
       [ 0.741,  0.914,  0.287,  0.472],
       [ 0.146,  0.23 ,  0.787,  0.96 ],
       [ 0.242,  0.487,  0.787,  0.971],
       [ 0.503,  0.572,  0.787,  0.972],
       [ 0.588,  0.715,  0.787,  0.972],
       [ 0.726,  0.857,  0.787,  0.969],
       [ 0.869,  0.914,  0.787,  0.97 ]])

In [105]:
x.sort(axis=0)
x # min en max waarden voor x en y

array([[ 0.041,  0.23 , -0.001,  0.277],
       [ 0.146,  0.487,  0.03 ,  0.422],
       [ 0.242,  0.557,  0.094,  0.472],
       [ 0.503,  0.572,  0.095,  0.764],
       [ 0.588,  0.693,  0.287,  0.777],
       [ 0.607,  0.701,  0.594,  0.779],
       [ 0.631,  0.715,  0.594,  0.96 ],
       [ 0.712,  0.857,  0.787,  0.969],
       [ 0.726,  0.914,  0.787,  0.97 ],
       [ 0.741,  0.914,  0.787,  0.971],
       [ 0.745,  0.914,  0.787,  0.971],
       [ 0.869,  0.914,  0.787,  0.972],
       [ 0.968,  0.981,  0.787,  0.972]])

In [76]:
blocks[5]

{'number': 5,
 'type': 0,
 'bbox': (112.47360229492188,
  511.299072265625,
  502.7896423339844,
  583.3207397460938),
 'lines': [{'spans': [{'size': 16.5,
     'flags': 16,
     'font': 'Aller-Bold',
     'color': 5487851,
     'ascender': 1.0369999408721924,
     'descender': -0.3179999887943268,
     'text': 'Dienstverlening, participatie en organisatie',
     'origin': (112.535400390625, 578.07373046875),
     'bbox': (112.535400390625,
      560.9632568359375,
      443.4923095703125,
      583.3207397460938)}],
   'wmode': 0,
   'dir': (1.0, 0.0),
   'bbox': (112.535400390625,
    560.9632568359375,
    443.4923095703125,
    583.3207397460938)},
  {'spans': [{'size': 40.0,
     'flags': 0,
     'font': 'Aller-Regular',
     'color': 5487851,
     'ascender': 1.0369999408721924,
     'descender': -0.3179999887943268,
     'text': 'Met en voor inwoners',
     'origin': (112.47360229492188, 552.779052734375),
     'bbox': (112.47360229492188,
      511.299072265625,
      502.78964

In [ ]:
[elem['text'] for elem in elements if elem['tag'] == primary_heading]
[elem['text'] for elem in elements if elem['tag'] == '<h3>']

['Met en voor inwoners',
 'Bouwen  om te wonen',
 'Groen doen we samen',
 'Economisch vitale gemeente',
 'Bereikbaar en bereisbaar',
 'De basis op orde',
 'Vitale wijken',
 'Positieve gezondheid',
 'Financiële leidende principes']

In [3]:
[elem for elem in elements if elem['page_num'] ==18]

NameError: name 'elements' is not defined

verschil in pixels in eigen beleidsdocument nooit groter dan 15 als het tussen een heading en een paragraaf is.

In [8]:
b_

[{'size': 27.0,
  'flags': 20,
  'font': 'KievitPro-Bold',
  'color': 2236191,
  'ascender': 0.9380000233650208,
  'descender': -0.25,
  'text': 'Inhoud',
  'origin': (361.41729736328125, 56.5201416015625),
  'bbox': (361.41729736328125,
   31.194141387939453,
   444.9822998046875,
   63.2701416015625)},
 {'size': 12.0,
  'flags': 4,
  'font': 'KievitPro-Regular',
  'color': 2236191,
  'ascender': 0.9380000233650208,
  'descender': -0.25,
  'text': 'Voorwoord: Aan de slag voor Aalsmeer! ',
  'origin': (361.41729736328125, 85.32012939453125),
  'bbox': (361.41729736328125,
   74.06413269042969,
   558.6470947265625,
   88.32012939453125)},
 {'size': 12.0,
  'flags': 4,
  'font': 'KievitPro-Regular',
  'color': 2236191,
  'ascender': 0.9380000233650208,
  'descender': -0.25,
  'text': 'Gemeentebestuur                                     ',
  'origin': (361.41729736328125, 114.1201171875),
  'bbox': (361.41729736328125,
   102.86412048339844,
   551.2445678710938,
   117.1201171875)},
 {'